In [2]:
# you are given data in the format of tagged_data.csv
#As there are 8x8 grid on each, for cell which is not tagged, there will be no entry in csv
# convert it to below using python code
# image_name, row,col,label

# e.g. 
# image_name, row,col,label
# image1.png,3,3,Bat

# Save new python file as tag_to_features_conversion.ipynb


Wrote 380 rows to tagged_data_converted.csv


## Fill missing 8x8 grid cells with `no_object`
The cell below reads `tagged_data_converted.csv` (flattened tags) and produces `tagged_data_converted_full.csv` containing every `(row,col)` pair (0..7) for each image. Missing entries are labeled `no_object`.

In [7]:
import pandas as pd
from pathlib import Path

base = Path('.')
# Prefer the already-converted flattened CSV if present
# flat_csv = base / 'tagged_data_converteds.csv'
orig_csv = base / 'tagged_data.csv'

# if flat_csv.exists():
#     df_flat = pd.read_csv(flat_csv)
# else:
# fallback: build flattened frame from original tagged_data.csv
df_raw = pd.read_csv(orig_csv)
rows = []
cell_cols = [c for c in df_raw.columns if c.startswith('cell_') and c.endswith('_row')]
max_cells = 0
for c in cell_cols:
    try:
        n = int(c.split('_')[1])
        if n > max_cells: max_cells = n
    except Exception:
        pass
if max_cells == 0: max_cells = 64
for _, r in df_raw.iterrows():
    img = r.get('imagename') or r.get('image_name') or r.get('image')
    for i in range(1, max_cells+1):
        rk=f'cell_{i}_row'; ck=f'cell_{i}_column'; tk=f'cell_{i}_object_tag'
        if rk in r.index and ck in r.index and tk in r.index:
            rv=r[rk]; cv=r[ck]; tv=r[tk]
            if pd.notna(rv) and pd.notna(cv) and pd.notna(tv) and str(tv).strip()!='':
                try:
                    rows.append({'image_name':img,'row':int(float(rv)),'col':int(float(cv)),'label':str(tv).strip()})
                except Exception:
                    pass
df_flat = pd.DataFrame(rows, columns=['image_name','row','col','label'])

# Now expand each image to full 8x8 grid, filling missing cells with 'no_object'
out_rows = []
for img, grp in df_flat.groupby('image_name'):
    # build lookup of existing labels
    lookup = {(int(r.row), int(r.col)): r.label for _, r in grp.iterrows()}
    for rr in range(8):
        for cc in range(8):
            lab = lookup.get((rr, cc), 'no_object')
            out_rows.append({'image_name': img, 'row': rr+1, 'col': cc+1, 'tile_number': rr*8 + cc + 1, 'label': lab})

out_df = pd.DataFrame(out_rows, columns=['image_name','row','col','tile_number','label'])
out_path = base / 'tagged_data_converted_full.csv'
out_df.to_csv(out_path, index=False)
out_df.head(12)

,image_name,row,col,tile_number,label
0,image1.png,1,1,1,no_object
1,image1.png,1,2,2,no_object
2,image1.png,1,3,3,no_object
3,image1.png,1,4,4,no_object
4,image1.png,1,5,5,no_object
5,image1.png,1,6,6,no_object
6,image1.png,1,7,7,no_object
7,image1.png,1,8,8,no_object
8,image1.png,2,1,9,no_object
9,image1.png,2,2,10,no_object
